In [1]:
#!pip uninstall matplotlib
# !pip install parse
!pip install plotly --upgrade

     |████████████████████████████████| 15.2 MB 203 kB/s eta 0:00:01
  Using cached tenacity-8.0.1-py3-none-any.whl (24 kB)


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import plotly.express as px
import pandas as pd
import plotly
import datetime as dt
from parse import *

In [3]:
colab_datadir = '/content/drive/MyDrive/sharedCodeBase/FamilyAnalysis/'
local_datadir = './'
datadir = local_datadir
import sys
sys.path.append(datadir)

# Advanced Time Line

In [4]:
EXCEL_FILE = "family1/figures_tag0/All_time_advanced.xlsx"   #每次改这里
# read excel
ori_df = pd.read_excel(datadir+EXCEL_FILE)

In [5]:
def label_color(zone):
  index = int(zone[0])
  color = 15 * (index + 1)
  if zone.endswith("非常驻"):
    return color - 5
  return color

df = ori_df.copy()
print(df)
df['Color'] = df.apply(lambda zones: label_color(zones["Zone"]), axis=1) #这一列只是用于Continunous Color的时候有用
df['Dates'] = df['Start'].dt.date
df['ST'] = df['Start'].dt.time
df['ET'] = df['End'].dt.time
print(df)

    Zone               Start                 End  Period
0    0常驻 2022-02-07 09:17:00 2022-02-07 09:18:00       1
1    0常驻 2022-02-07 09:27:00 2022-02-07 09:34:00       7
2    0常驻 2022-02-07 09:38:00 2022-02-07 09:44:00       6
3    0常驻 2022-02-07 09:55:00 2022-02-07 09:56:00       1
4    0常驻 2022-02-07 10:00:00 2022-02-07 10:01:00       1
..   ...                 ...                 ...     ...
667  3常驻 2022-02-10 11:59:00 2022-02-10 12:28:00      29
668  3常驻 2022-02-03 19:16:00 2022-02-03 20:25:00      69
669  3常驻 2022-02-03 20:29:00 2022-02-03 20:32:00       3
670  3常驻 2022-02-03 20:41:00 2022-02-03 20:42:00       1
671  3常驻 2022-02-09 11:54:00 2022-02-09 12:12:00      18

[672 rows x 4 columns]
    Zone               Start                 End  Period  Color       Dates  \
0    0常驻 2022-02-07 09:17:00 2022-02-07 09:18:00       1     15  2022-02-07   
1    0常驻 2022-02-07 09:27:00 2022-02-07 09:34:00       7     15  2022-02-07   
2    0常驻 2022-02-07 09:38:00 2022-02-07 09:44:00       

In [6]:
#需要改最后一行，生成图片名称

# IMPORTANT ! 
# Adjust day time : add a fake start time and a fake end time
fake_s = df.iloc[0]
fake_e = df.iloc[0]
fake_s['Period'] =0
fake_s['ST'] = dt.time(6,0,0)
fake_s['ET'] = dt.time(6,0,0)
fake_e['Period'] =0
fake_e['ST'] = dt.time(23,59,59)
fake_e['ET'] = dt.time(23,59,59)
#print(fake_s)
df=df.append(fake_s)
df=df.append(fake_e)


# prepare for figure
reference = dt.datetime(2022,1,1,0,0)
# combine time all in the same day
df["Start"] = df.apply(lambda x :  dt.datetime.combine(reference,x["ST"]), axis=1 )
df["End"] = df.apply(lambda x :  dt.datetime.combine(reference,x["ET"]), axis=1 )

# draw overview time line
zones = df["Zone"]
datetime_series_s = df["Start"]
datetime_series_e = df["End"]
color = df["Color"]
Dates = df["Dates"]
information = df["Zone"]
#-----------------------------------
# Continunous Colors 连续颜色分布
# fig = px.timeline(df, x_start=datetime_series_s, x_end=datetime_series_e, y=Dates, color="Color", hover_name = information,opacity=0.9)
#-----------------------------------
# Discrete Colors 1 彩色分布
# fig = px.timeline(df, x_start=datetime_series_s, x_end=datetime_series_e, y=Dates, color="Zone", hover_name = information,opacity=0.9,
#                   color_discrete_sequence=px.colors.qualitative.G10)
#-----------------------------------
# Discrete Colors 2 渐变离散分布
# fig = px.timeline(df, x_start=datetime_series_s, x_end=datetime_series_e, y=Dates, color="Zone", hover_name = information,opacity=0.9,
#                   color_discrete_sequence= px.colors.sequential.Plasma_r)
#-----------------------------------
# Discrete Colors 3 手动定义分布
# 资源 https://www.heavy.ai/blog/12-color-palettes-for-telling-better-stories-with-your-data
# 资源2 https://colorhunt.co/
blue2red = ["#1984c5", "#22a7f0", "#63bff0", "#a7d5ed", "#e2e2e2", "#e1a692", "#de6e56", "#e14b31", "#c23728"]
pink2foam = ["#54bebe", "#76c8c8", "#98d1d1", "#badbdb", "#dedad2", "#e4bcad", "#df979e", "#d7658b", "#c80064"]
grid2red = ["#d7e1ee", "#cbd6e4", "#bfcbdb", "#b3bfd1", "#a4a2a8", "#df8879", "#c86558", "#b04238", "#991f17"]

self_defined_color_discrete_map = {}
order = 0
for zone in zones:
  if zone in self_defined_color_discrete_map :
    continue
  if zone.endswith("常驻"):
    self_defined_color_discrete_map[zone] = blue2red[order]
    order += 1

fig = px.timeline(df, x_start=datetime_series_s, x_end=datetime_series_e, y=Dates,
                  color="Zone", hover_name = information,opacity=0.9,
                  color_discrete_map={"0常驻": "#1984c5",
                                      "0非常驻": "#22a7f0",
                                      "1常驻": "#63bff0",
                                      "1非常驻": "#a7d5ed",
                                      "2常驻": "#e1a692",
                                      "2非常驻": "#e2e2e2",
                                      "3常驻": "#c23728",
                                      "3非常驻": "#de6e56"})

## 另一个写法
# fig = px.timeline(df, x_start=datetime_series_s, x_end=datetime_series_e, y=Dates, color="Zone", hover_name = information,opacity=0.9,
#                   color_discrete_map=self_defined_color_discrete_map)

#-----------------------------------

fig.update_yaxes(autorange="reversed")
fig.update_layout(title_font_size=42, font_size=18, title_font_family="Arial")

plotly.offline.plot(fig, filename=datadir+"Timelines/"+"Advanced_Time_Overview_Gantt.html")

/tmp/ipykernel_69867/2803054753.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_s['Period'] =0
/tmp/ipykernel_69867/2803054753.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_s['ST'] = dt.time(6,0,0)
/tmp/ipykernel_69867/2803054753.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_s['ET'] = dt.time(6,0,0)
/tmp/ipykernel_69867/2803054753.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

'./Timelines/Advanced_Time_Overview_Gantt.html'

# Old Version

In [8]:
# EXCEL_FILE = "family1/figures_tag0/All_time_advanced.xlsx"   #每次改这里family1_figures_tag0/All_time.xlsx
# # read excel
# df = pd.read_excel(datadir+EXCEL_FILE)
# ori_df = df.copy()
# print(df)

## Overview Time Line Gantt

In [9]:
# add new columns
# def label_name(zones):
#   if zones['Zone'] == 0:
#     return 'Zone0'
#   if zones['Zone'] == 1:
#     return 'Zone1'
#   if zones['Zone'] == 2:
#     return 'Zone2'
#   if zones['Zone'] == 3:
#     return 'Zone3'
#   if zones['Zone'] == 4:
#     return 'Zone4'
#   if zones['Zone'] == 5:
#     return 'Zone5'
#   if zones['Zone'] == 6:
#     return 'Zone6'
#   if zones['Zone'] == 7:
#     return 'Zone7'
#   if zones['Zone'] == 8:
#     return 'Zone8'
#   return 'Zone9'

# def label_color(zones):
#   if zones['Zone'] == 0:
#     return 5
#   if zones['Zone'] == 1:
#     return 10
#   if zones['Zone'] == 2:
#     return 15
#   if zones['Zone'] == 3:
#     return 20
#   if zones['Zone'] == 4:
#     return 25
#   if zones['Zone'] == 5:
#     return 30
#   if zones['Zone'] == 6:
#     return 35
#   if zones['Zone'] == 7:
#     return 40
#   if zones['Zone'] == 8:
#     return 45
#   return 50

# df['Name'] = df.apply (lambda zones: label_name(zones), axis=1)
# df['Color'] = df.apply(lambda zones: label_color(zones), axis=1)
# print(df)

In [10]:
# # draw overview time line
# zones = df["Zone"]
# datetime_series_s = df["Start"]
# datetime_series_e = df["End"]
# color = df["Color"]
# information = df["Name"]

# fig = px.timeline(df, x_start=datetime_series_s, x_end=datetime_series_e, y=zones, color=color, hover_name = information)
# fig.update_yaxes(autorange="reversed")
# fig.update_layout(title_font_size=42, font_size=18, title_font_family="Arial")

# plotly.offline.plot(fig, filename=datadir+"Timelines/"+"Time_Overview_Gantt.html")

## Daily Time Line For Each Zone

In [11]:
# df = ori_df.copy()
# df['Dates'] = df['Start'].dt.date
# df['ST'] = df['Start'].dt.time
# df['ET'] = df['End'].dt.time
# df['Name'] = df.apply(lambda x: x['Dates'].strftime("%y-%m-%d"), axis=1)

### Define colors

Please only use one method

#### First Method

In [12]:
# days = df['Dates'].unique()
# colors = {}
# for i in range(len(days)):
#   #colors[days[i]] = (i+1)*5 # different random colors
#   colors[days[i]] = 70 # same color
# df['Color'] = df.apply(lambda x: colors[x['Dates']], axis=1)
# print(df)

#### Second Method

In [13]:
# import math
# df['Color'] = df.apply(lambda x: math.sqrt(x['Period']), axis=1)
# print(df)

In [14]:
# zones = [y for _, y in df.groupby('Zone', as_index=False)]
# print("Example of Zone 0")
# zones[0]


### 开始作图

In [15]:
# # 开始画图
# index = 0
# for zone in zones:
#   zone_info = "Zone"+str(index)
#   index +=1
#   # IMPORTANT ! 
#   # Adjust day time : add a fake start time and a fake end time
#   fake_s = zone.iloc[0]
#   fake_e = zone.iloc[0]
#   fake_s['Period'] =0
#   fake_s['ST'] = dt.time(6,0,0)
#   fake_s['ET'] = dt.time(6,0,0)
#   fake_e['Period'] =0
#   fake_e['ST'] = dt.time(23,59,59)
#   fake_e['ET'] = dt.time(23,59,59)
#   #print(fake_s)
#   zone=zone.append(fake_s)
#   zone=zone.append(fake_e)

#   # prepare for figure
#   name = zone["Name"]
#   reference = dt.datetime(2022,1,1,0,0)
#   # combine time all in the same day
#   zone["Start"] = zone.apply(lambda x :  dt.datetime.combine(reference,x["ST"]), axis=1 )
#   zone["End"] = zone.apply(lambda x :  dt.datetime.combine(reference,x["ET"]), axis=1 )
#   datetime_series_s =zone["Start"]
#   datetime_series_e = zone["End"]
#   color = zone["Color"]
#   information = zone["Dates"]
#   fig = px.timeline(df, x_start=datetime_series_s, x_end=datetime_series_e, y=name, color=color, hover_name = information)
#   fig.update_yaxes(autorange="reversed")
#   fig.update_layout(title_font_size=42, font_size=18, title_font_family="Arial")

#   plotly.offline.plot(fig, filename=datadir+"Timelines/"+"TimeLine_"+zone_info+".html")